In [3]:
from scipy.io import loadmat
from scipy.io import savemat
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys

import random
from tqdm import tqdm

In [4]:
if "google.colab" in sys.modules:
    root_dir = os.path.join(
        os.getcwd(), "drive", "MyDrive", "Colab Notebooks"
    )
    data_dir = os.path.join(root_dir, "data")
else:
    root_dir = os.getcwd()
    data_dir = os.path.join(root_dir, "data")

In [5]:
dir = os.path.join(data_dir, "SPST_perfect")

In [6]:
file = loadmat(os.path.join(dir, "SPSTdata_2_pol-60.mat"))

In [7]:
maps = file["AcombineAll"][0]
azimuts = file["Fa_real"][0]
elevations = file["Fb_real"][0]
nom_intensities = file["Max4"][0]
X = file["Xnom_dumARRAY"][0]
Y = file["Ynom_dumARRAY"][0]
df = pd.DataFrame(
    {
        "map": maps,
        "azimut": azimuts,
        "elevation": elevations,
        "nom_intensity": nom_intensities,
        "X": X,
        "Y": Y,
    }
)

In [8]:
def get_index(X, Y, x, y):
    for i in range(len(X)):
        if X[i] == x and Y[i] == y:
            return i
    return -1

def get_neighbours(map, x, y):
    neighbours = []
    if x > 0:
        neighbours.append((x - 1, y))
    if x < len(map) - 1:
        neighbours.append((x + 1, y))
    if y > 0:
        neighbours.append((x, y - 1))
    if y < len(map) - 1:
        neighbours.append((x, y + 1))
    neighbours = [n for n in neighbours if not np.isnan(map[n[0]][n[1]])]
    return neighbours

def get_maps(df, i, j, forbiden, len_map):
    values = []
    X = []
    Y = []
    for l in range(len(df)):
        # ignore the additional values in the center
        if df["X"][l] not in forbiden and df["Y"][l] not in forbiden:
            values.append(df["map"][l][i][j])
            X.append(df["X"][l])
            Y.append(df["Y"][l])
    coords = []
    for i in range(len(X)):
        coords.append((X[i], Y[i]))
    
    map = np.zeros((len_map, len_map))
    map_coords = np.zeros((len_map, len_map), dtype=object)
    for x in range(len_map):
        for y in range(len_map):
            x_real = x * 20 + 7
            y_real = y * 20 + 7

            if (x_real, y_real) in coords:
                index = coords.index((x_real, y_real))
                map[x][y] = values[index]
                map_coords[x][y] = (x_real, y_real)
            elif (x_real + 1, y_real) in coords:
                index = coords.index((x_real + 1, y_real))
                map[x][y] = values[index]
                map_coords[x][y] = (x_real + 1, y_real)
            elif (x_real - 1, y_real) in coords:
                index = coords.index((x_real - 1, y_real))
                map[x][y] = values[index]
                map_coords[x][y] = (x_real - 1, y_real)
            elif (x_real, y_real + 1) in coords:
                index = coords.index((x_real, y_real + 1))
                map[x][y] = values[index]
                map_coords[x][y] = (x_real, y_real + 1)
            elif (x_real, y_real - 1) in coords:
                index = coords.index((x_real, y_real - 1))
                map[x][y] = values[index]
                map_coords[x][y] = (x_real, y_real - 1)
            elif (x_real + 1, y_real + 1) in coords:
                index = coords.index((x_real + 1, y_real + 1))
                map[x][y] = values[index]
                map_coords[x][y] = (x_real + 1, y_real + 1)
            elif (x_real - 1, y_real - 1) in coords:
                index = coords.index((x_real - 1, y_real - 1))
                map[x][y] = values[index]
                map_coords[x][y] = (x_real - 1, y_real - 1)
            elif (x_real + 1, y_real - 1) in coords:
                index = coords.index((x_real + 1, y_real - 1))
                map[x][y] = values[index]
                map_coords[x][y] = (x_real + 1, y_real - 1)
            elif (x_real - 1, y_real + 1) in coords:
                index = coords.index((x_real - 1, y_real + 1))
                map[x][y] = values[index]
                map_coords[x][y] = (x_real - 1, y_real + 1)
            else:
                map[x][y] = np.nan
                map_coords[x][y] = (x_real, y_real)
    return map, map_coords

def densify_map(map):
    len_map = len(map)
    # double the density of the map by intercalating nan values
    map2 = np.zeros((len_map * 2 - 1, len_map * 2 - 1))
    for x in range(len_map):
        for y in range(len_map):
            map2[x * 2][y * 2] = map[x][y]
            if y < len_map - 1:
                map2[x * 2][y * 2 + 1] = np.nan
            if x < len_map - 1:
                map2[x * 2 + 1][y * 2] = np.nan
            if x < len_map - 1 and y < len_map - 1:
                map2[x * 2 + 1][y * 2 + 1] = np.nan

    # while there are nan values, interpolate them
    while np.isnan(map2).any():
        for x in range(len(map2)):
            for y in range(len(map2)):
                if np.isnan(map2[x][y]):
                    neighbours = get_neighbours(map2, x, y)
                    if len(neighbours) > 0:
                        map2[x][y] = np.mean([map2[n[0]][n[1]] for n in neighbours])
    return map2

In [24]:
from multiprocessing import Lock, Process, Queue, current_process
import queue
import logging
from tqdm import tqdm

len_map = 26
forbidden = [196, 197, 216, 217, 236, 237, 251, 255, 256, 262, 276, 277, 297, 317]

# Number of tasks for your loop
len_map_big = len(df["map"][0])
total_tasks = len_map_big * len_map_big

maps = np.zeros((total_tasks, len_map, len_map))
maps_coords = np.zeros((total_tasks, len_map, len_map), dtype=object)
nom_coords = np.zeros((total_tasks), dtype=object)

def process_map(i, j):
    print("Process map", i, j)

    map, map_coords = get_maps(df, i, j, forbidden, len_map)
    # transpose the maps
    map = map.T
    map_coords = map_coords.T
    # reverse the y axis
    map = map[::-1]
    map_coords = map_coords[::-1]

    ind = i * len_map_big + j
    maps[ind] = map
    maps_coords[ind] = map_coords
    nom_coords[ind] = (i, j)
    return

def do_work(tasks_to_accomplish, tasks_that_are_done):
    print("Process started")
    while True:
        try:
            i, j = tasks_to_accomplish.get_nowait()
            print("Process map", i, j)
        except queue.Empty:
            print("Queue empty")
            break
        else:
            print("Process map", i, j)
            process_map(i, j)
            tasks_that_are_done.put(f"Task {i} {j} is done by {current_process().name}")
    return True

num_processes = 8
processes = []

tasks_queue = Queue()
results_queue = Queue()

for i in range(len_map_big):
    for j in range(len_map_big):
        tasks_queue.put((i, j))

# print the size of the queue
print("Queue size:", tasks_queue.qsize())

for w in range(num_processes):
    p = Process(target=do_work, args=(tasks_queue, results_queue))
    processes.append(p)
    p.start()

print("Processes size:", len(processes))

for p in processes:
    p.join()

print("All work completed.")

Queue size: 262144
Processes size: 8
All work completed.


In [1]:
import multiprocessing

print("Number of cpu : ", multiprocessing.cpu_count())

Number of cpu :  12


In [19]:
len_map = 26

forbiden = [196, 197, 216, 217, 236, 237, 251, 255, 256, 262, 276, 277, 297, 317]

# Number of iterations for your loop
len_map_big = len(df["map"][0])
total_tasks = len_map_big * len_map_big

maps = np.zeros((total_tasks, len_map, len_map))
maps_coords = np.zeros((total_tasks, len_map, len_map), dtype=object)
nom_coords = np.zeros((total_tasks), dtype=object)

# Initialize the progress bars
progress_bar = tqdm(desc='Process', total=total_tasks, unit='task', dynamic_ncols=True)

for i in range(len(df["map"][0])):
    for j in range(len(df["map"][0][0])):
        map, map_coords = get_maps(df, i, j, forbiden, len_map)
        # transpose the maps
        map = map.T
        map_coords = map_coords.T
        #reverse the y axis
        map = map[::-1]
        map_coords = map_coords[::-1]
        ind = i * len_map_big + j
        maps[ind] = map
        maps_coords[ind] = map_coords
        nom_coords[ind] = (i, j)
        progress_bar.update(1)

# Close the progress bar
progress_bar.close()

Process:   0%|          | 0/262144 [00:00<?, ?task/s]

KeyboardInterrupt: 

In [8]:
ind = random.randint(0, len(df))
map = maps[ind]
map_coords = maps_coords[ind]
map2 = densify_map(map)

# plot the map
plt.imshow(map, cmap="jet")
plt.colorbar()
plt.show()

# plot the map
plt.imshow(map2, cmap="jet")
plt.colorbar()
plt.show()

plt.scatter(
    [map_coords[x][y][0] for x in range(len(map_coords)) for y in range(len(map_coords))],
    [map_coords[x][y][1] for x in range(len(map_coords)) for y in range(len(map_coords))],
    c=[map[x][y] for x in range(len(map)) for y in range(len(map))],
    cmap="jet",
    s=100,
    marker="s",
)
plt.colorbar()
plt.show()

IndexError: list index out of range

In [ ]:
dict = {name: col.values for name, col in df.items()}
savemat(os.path.join(dir, "SPSTdata_2_pol-60.mat"), dict)